In [2]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [3]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo

## Dataset

In [4]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1.5cm.csv')

In [5]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [6]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelo

In [7]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(100,100,100,100,100), EPOCHS_NOCHANGE = 20)

## Estimação de Parâmatros

In [8]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

## Treino

In [9]:
mlp.set_model(**params)

In [10]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [11]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

### Erro juntas

In [12]:
metrica.predict_joint(x = x_train)

array([[ 65.63728466,  37.64542696,  84.25842577,  46.86745838],
       [ 96.175419  ,  95.18336342,  83.34731067,  60.30910712],
       [ 35.55518928,  83.57556657,  83.73180339,  36.2507363 ],
       ...,
       [ 93.95269506, 119.35308256,  10.48478097,  35.50424211],
       [ 19.7962104 ,  36.55654538,  85.09270133,  35.67230024],
       [ 35.31630349,  36.16963876,  47.16363729,  -0.93332684]])

In [13]:
metrica.mse_joint(x = x_train, y = y_train)

array([0.55269527, 1.56803829, 5.15688068, 2.22570988])

### Erro cartesiano

In [14]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo",Elos)

metrica.robotic(robo = robo)

In [15]:
a = metrica.predict_operacional(x = x_train)
a

[[0.004045669164077787,
  0.41251172397866087,
  -0.91094330786343,
  -14.677198386877034,
  -0.0018320234516133523,
  0.9109522916048639,
  0.4125076558197674,
  6.646359491649077,
  0.9999901380769151,
  0.0,
  0.004441142748486155,
  -13.995125528461017],
 [0.9463130531156254,
  -0.10757283571511686,
  -0.30482731262009966,
  -24.61870038735419,
  0.10239173672258732,
  0.9941972063007462,
  -0.032982529234791655,
  -2.663760665896923,
  0.30660648680991054,
  0.0,
  0.9518363631665289,
  12.898677575381567],
 [0.3425648391143907,
  0.8135557875447028,
  -0.4698683981234001,
  -17.7532013873492,
  -0.4792809977564513,
  0.5814868704902273,
  0.6573908613884606,
  24.83842794928132,
  0.8080464443286117,
  0.0,
  0.589118785821576,
  2.6680896753282015],
 [0.1846730371536904,
  -0.3402061986233704,
  0.9220388341966229,
  10.99039347727959,
  0.06681220227620793,
  0.9403508613375307,
  0.33358115535618504,
  3.976175425587549,
  -0.9805263887195667,
  0.0,
  0.19638737491133496,
  1

In [16]:
metrica.mse_operacional(x = x_train, y = y_train)

array([[ 1.88515878e+01,  4.04273604e+01,  7.20352768e+00,
         2.23408545e+00, -1.57079633e+00,  2.04197122e+00],
       [-1.85532355e+00,  2.12064452e+01,  4.87628218e+01,
        -1.11022302e-16, -3.14159265e-01, -1.48352986e+00],
       [ 2.82147241e+01,  1.97561625e+01,  3.84815287e+01,
        -1.11022302e-16, -9.42477796e-01, -2.53072742e+00],
       ...,
       [ 2.53837717e+00, -2.90137839e+01,  5.32876091e+01,
         0.00000000e+00,  9.42477796e-01, -1.48352986e+00],
       [ 4.15469246e+01,  1.51218439e+01,  3.46111725e+00,
         3.14159265e+00, -1.36135682e+00,  3.49065850e-01],
       [ 4.07738005e+01,  2.85501225e+01,  1.30953137e+01,
         3.14159265e+00, -1.36135682e+00,  6.10865238e-01]])